<a href="https://colab.research.google.com/github/Eliokay/Visualizing-US-Health-Insurance-Coverage/blob/main/Eloho's_Grouped_Healthcare_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import geopandas as gpd
from vega_datasets import data
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/originalDataset.csv')
df.shape

(80818, 59)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# df.dropna(subset = 'HCOVANY',inplace = True)
# df.shape

(80818, 59)

In [ ]:
df.head(15)

In [ ]:
adults = df[df['AGE']>=18]
adults2022 = adults[adults['YEAR'] == 2022]
adults2022.shape

(64960, 59)

In [ ]:
adults['COUNTYFIP'].value_counts()

33    17952
0     14381
53     7216
61     6522
63     4525
11     3824
35     2466
67     2235
73     2017
57     1942
77     1880
Name: COUNTYFIP, dtype: int64

In [ ]:
bycategory = adults.groupby(['COUNTYFIP','CITY',"SEX", "AGE", "MARRNO", "RACE", "RACED", "BPL", "BPLD", "CITIZEN", "LANGUAGE", "LANGUAGED", "SPEAKENG", "RACAMIND", "RACASIAN", "RACBLK", "RACPACIS", "RACWHT", "RACOTHER", "HCOVANY", "HCOVPRIV", "HINSEMP", "HINSPUR", "HINSTRI", "HCOVPUB", "HINSCAID", "HINSCARE", "HINSVA", "HINSIHS", "SCHOOL", "EDUC", "EDUCD", "SCHLTYPE", "DEGFIELD", "DEGFIELDD", "EMPSTAT", "EMPSTATD", "CLASSWKR", "CLASSWKRD", "OCC", "INCTOT", "FTOTINC", "POVERTY", "VETDISAB", "DIFFREM", "DIFFPHYS", "VETSTAT", "VETSTATD"],as_index=False).count()
bycategory.head()

,COUNTYFIP,CITY,SEX,AGE,MARRNO,RACE,RACED,BPL,BPLD,CITIZEN,...,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,STRATA,GQ,PERNUM,PERWT
0,0,0,1,18,0,1,100,4,400,0,...,1,1,1,1,1,1,1,1,1,1
1,0,0,1,18,0,1,100,5,500,0,...,1,1,1,1,1,1,1,1,1,1
2,0,0,1,18,0,1,100,6,600,0,...,1,1,1,1,1,1,1,1,1,1
3,0,0,1,18,0,1,100,6,600,0,...,1,1,1,1,1,1,1,1,1,1
4,0,0,1,18,0,1,100,6,600,0,...,1,1,1,1,1,1,1,1,1,1


In [ ]:
byeducation = adults.groupby(['EDUC'],as_index=False)['HCOVANY'].mean()
byeducation['HCOVANY'] = byeducation['HCOVANY'] -1

byeducation.loc[byeducation['EDUC'] == 0o0, 'EDUC'] = 'No Schooling'
byeducation.loc[byeducation['EDUC'] == 0o1, 'EDUC'] = 'Nursery school to grade 4'
byeducation.loc[byeducation['EDUC'] == 0o2, 'EDUC'] = 'Grade 5, 6, 7, or 8'
byeducation.loc[byeducation['EDUC'] == 0o3, 'EDUC'] = 'Grade 9'
byeducation.loc[byeducation['EDUC'] == 0o4, 'EDUC'] = 'Grade 10'
byeducation.loc[byeducation['EDUC'] == 0o5, 'EDUC'] = 'Grade 11'
byeducation.loc[byeducation['EDUC'] == 0o6, 'EDUC'] = 'Grade 12'
byeducation.loc[byeducation['EDUC'] == 0o7, 'EDUC'] = '1 year of college'
# byeducation.loc[byeducation['EDUC'] == 0o8, 'EDUC'] = '2 year of college'
# byeducation.loc[byeducation['EDUC'] == 0o9, 'EDUC'] = '3 year of college'
byeducation.loc[byeducation['EDUC'] == 10, 'EDUC'] = '4 years of college'
byeducation.loc[byeducation['EDUC'] == 11, 'EDUC'] = '5+ years of college'
byeducation.loc[byeducation['EDUC'] == 99, 'EDUC'] = 'Missing'


byeducation.head(60)

,EDUC,HCOVANY
0,No Schooling,0.813291
1,Nursery school to grade 4,0.746914
2,"Grade 5, 6, 7, or 8",0.729138
3,Grade 9,0.815725
4,Grade 10,0.872763
5,Grade 11,0.885529
6,Grade 12,0.918194
7,1 year of college,0.946356
8,8,0.954197
9,4 years of college,0.971030


In [ ]:
base = alt.Chart(byeducation).mark_bar().encode(
    alt.Y('EDUC:N', scale=alt.Scale(domain=['No Schooling','Grade 5, 6, 7, or 8', 'Grade 9', '1 year of college', '4 years of college', '5+ years of college']), title=""),
    alt.X('HCOVANY:Q', axis=alt.Axis(format='%'), title="Health Insurance Coverage"),
    text = "HCOVANY",
).properties(width=500,height=500, title='Education Level in Relation to Health Insurance Coverage')
base.configure_title(
     fontSize=25,
     anchor='middle',
).configure_header(
    labelFontSize=20
).configure_axis(
    titleFontSize=16,
    labelFontSize=18,
    gridOpacity=0
)



alt.Chart(...)

In [ ]:
byFIPS = adults.groupby(['COUNTYFIP'],as_index=False)['HCOVANY'].mean()
byFIPS['HCOVANY'] = byFIPS['HCOVANY'] -1
byFIPS.tail(20)

,COUNTYFIP,HCOVANY
0,0,0.918921
1,11,0.949006
2,33,0.954991
3,35,0.954582
4,53,0.949972
5,57,0.952626
6,61,0.949249
7,63,0.952707
8,67,0.957494
9,73,0.941001


In [ ]:
countydf = gpd.read_file(data.us_10m.url,driver='TopoJSON')
countydf.head()

countydf['WAFIPS'] = countydf['id'].str.extract('(\d{2})$')

washington = countydf[countydf['id'].str.match('53\d*')]

byFIPS.dropna(subset = 'HCOVANY',inplace = True)
byFIPS['COUNTYFIP'] = byFIPS['COUNTYFIP'].astype(int)
washington['WAFIPS'] = washington['WAFIPS'].astype(int)

mapdata = byFIPS.merge(washington, left_on = 'COUNTYFIP', right_on='WAFIPS', how='outer')
# byFIPS.head()
# washington.head()
mapdata.head(12)

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,COUNTYFIP,HCOVANY,id,geometry,WAFIPS
0,0.0,0.918921,53000,"POLYGON ((-122.65544 48.41032, -122.65544 48.4...",0
1,11.0,0.949006,53011,"POLYGON ((-122.24626 46.05383, -122.24985 45.5...",11
2,33.0,0.954991,53033,"MULTIPOLYGON (((-122.50110 47.33010, -122.4759...",33
3,35.0,0.954582,53035,"MULTIPOLYGON (((-122.51905 47.57397, -122.5944...",35
4,53.0,0.949972,53053,"MULTIPOLYGON (((-122.42214 47.32044, -122.3359...",53
5,57.0,0.952626,53057,"MULTIPOLYGON (((-122.65544 48.41032, -122.6051...",57
6,61.0,0.949249,53061,"MULTIPOLYGON (((-121.00078 48.29537, -120.9218...",61
7,63.0,0.952707,53063,"POLYGON ((-117.43661 48.04720, -117.04178 48.0...",63
8,67.0,0.957494,53067,"POLYGON ((-122.69851 47.09698, -122.63749 46.9...",67
9,73.0,0.941001,53073,"MULTIPOLYGON (((-120.85361 49.00011, -120.7674...",73


In [ ]:
basemap=alt.Chart(washington).mark_geoshape().encode(color=alt.value('lightgray'))

bm2=basemap.encode(
    # alt.Color("HCOVANY:N", title='Health Insurance Coverage', scale = alt.Scale(scheme='blues')),
    color = alt.condition("datum.HCOVANY !== 'null'", 'HCOVANY:Q', alt.value('lightgray')),
    tooltip=['WAFIPS:N', 'HCOVANY:Q']
).transform_lookup(
    lookup='WAFIPS',
    from_=alt.LookupData(byFIPS, 'COUNTYFIP', ["HCOVANY"]),
).project(
    type='albersUsa'
).properties(
    width=800,
    height=600,
    title='Health Insurance Coverage By Counties in Washington'
)

# text = bm2.mark_text(baseline='middle').encode(
#     alt.Text('HCOVANY:N', format=".0%"),
#     color=alt.condition(
#         alt.datum.HCOVANY < .70,
#         alt.value('black'),
#         alt.value('white')
# ))

# bm2 + text
FinalMap = ((basemap + bm2).resolve_scale(color='independent'))
FinalMap
# bm2
FinalMap.configure_title(
     fontSize=30,
     anchor='middle').configure_header(
    labelFontSize=20
).configure_axis(
    titleFontSize=16,
    labelFontSize=10)


alt.LayerChart(...)

In [ ]:
bymarriage = adults.groupby(['MARRNO', 'RACE'],as_index=False)['HCOVANY'].mean()
bymarriage['HCOVANY'] = bymarriage['HCOVANY'] -1

bymarriage.loc[bymarriage['RACE'] == 1, 'RACE'] = 'White'
bymarriage.loc[bymarriage['RACE'] == 2, 'RACE'] = 'Black'
bymarriage.loc[bymarriage['RACE'] == 3, 'RACE'] = 'Native'
bymarriage.loc[bymarriage['RACE'] == 4, 'RACE'] = 'Chinese'
bymarriage.loc[bymarriage['RACE'] == 5, 'RACE'] = 'Japanese'
bymarriage.loc[bymarriage['RACE'] == 6, 'RACE'] = 'Pacific Islander'
bymarriage.loc[bymarriage['RACE'] == 7, 'RACE'] = 'Other Race'
bymarriage.loc[bymarriage['RACE'] == 8, 'RACE'] = 'Two major races'
bymarriage.loc[bymarriage['RACE'] == 9, 'RACE'] = 'Three or more major races'

bymarriage.head()



,MARRNO,RACE,HCOVANY
0,0,White,0.929606
1,0,Black,0.907407
2,0,Native,0.780142
3,0,Chinese,0.949227
4,0,Japanese,0.973333


In [ ]:
base=alt.Chart(bymarriage).mark_rect(
).encode(
    alt.Y('RACE:N', title=''),
    alt.X('MARRNO:O', title='Number of years Married'),
    alt.Color('HCOVANY:Q', legend = None, )
).properties(width = 800, height = 800, title="Health Insurance Coverage by Duration of Marriage and Race in Washington")

text = base.mark_text(baseline='middle', size=20).encode(
    alt.Text('HCOVANY:Q', format=".0%"),
    color=alt.condition(
        alt.datum.HCOVANY < .80,
        alt.value('black'),
        alt.value('white')
    )
)

Finalheatmap = (base + text)
Finalheatmap.configure_title(
     fontSize=25,
     anchor='middle',
).configure_header(
    labelFontSize=20
).configure_axis(
    titleFontSize=18,
    labelFontSize=18,
    gridOpacity=0.2

)

#SHOULD I JUST SELECT A FEW TO TALK ABOUT?

alt.LayerChart(...)